In [2]:
%load_ext autoreload
%autoreload 2

In [21]:
#import kelp_param
import kelp3d_objs as k3
import ipyparallel as ipp
import concurrent.futures as cf
import itertools as it
#from sympy import divisors
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize as opt
from scipy.interpolate import interp1d
import ipyvolume as ipv
import ipywidgets as ipw

import kelp_compute
import kelp_analyze

In [5]:
import sqlite3
import shutil
import os

### Define Constants

In [50]:
out = ipw.Output()

In [51]:
out

Output()

In [63]:
# Hard-coded parameters
rope_spacing = 20.0 # (m)
zmin, zmax = 0.0, 10.0 # (m)

# Light from above
I0 = 50.0 # (W/m^2)
phi_s = 0 # (rad)
theta_s = 0 # (rad)
decay = 0 # (?) - light (practically) only from directly above

# Calculated or otherwise boring parameters
xmin = -rope_spacing/2
xmax = rope_spacing/2
ymin = -rope_spacing/2
ymax = rope_spacing/2

# Set parameters
study_name = 'single'
a_water = 0.179
b = 0.219
kelp_dist='top-heavy'

data_dir = os.path.join(os.environ['SCRATCH'], 'kelp-results')

ns = 30
nz = 14
na = 14

num_scatters = 1
fd_flag = False
lis_options = "-i gmres -restart 10000 -tol 1e-4"

# Compute

shutil.rmtree(os.path.join(data_dir, study_name), ignore_errors=False)

base_dir, db_path, table_name = kelp_compute.create_table(study_name, data_dir)

with out:
    kelp_compute.kelp_calculate(
        base_dir, db_path, table_name,
        a_water, b,
        ns, nz, na,
        kelp_dist, num_scatters,
        fd_flag,
        lis_options
    )

# Retrieve

conn = sqlite3.connect(db_path)
res = kelp_analyze.query_results(conn, table_name, ns=ns, nz=nz, na=na)[0]
conn.close()
irrad = res['irrad'][:]

print("{:.2f}% nan".format(np.sum(np.isnan(irrad))/np.size(irrad)))

# Visualize

ipv.figure()
ipv.volshow(irrad)
ipv.show()

0.04% nan


/global/homes/o/oevans/local/miniconda3/envs/kelp/lib/python3.6/site-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in greater
